### [How to route between sub-chains](https://python.langchain.com/docs/how_to/routing/)

In [2]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [3]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Example setup

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model_name="gpt-4o-mini")
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

'Anthropic'

In [8]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

anthropic_chain = PromptTemplate.from_template(
    """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-4o-mini")

#### B. Using a custom function (Recommended)

In [9]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [10]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [11]:
full_chain.invoke({"question": "how do I use Anthropic?"})

AIMessage(content="As Dario Amodei told me, using Anthropic involves leveraging its AI models and tools designed for safe and efficient interactions. You would typically start by accessing the platform, which may include APIs or user interfaces, depending on the product offerings. After that, you can integrate the models into your applications or workflows, ensuring you follow best practices for safety and alignment as emphasized by the team. Additionally, exploring the documentation and guidelines provided by Anthropic will help you optimize your usage and ensure you're making the most of the capabilities available.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 47, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-

In [12]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you need to follow a few key steps:\n\n1. **Installation**: Start by installing the LangChain library. You can do this using pip:\n   ```bash\n   pip install langchain\n   ```\n\n2. **Choose a Language Model**: LangChain works with various language models. You can use models from OpenAI, Hugging Face, or other providers. Make sure to set up the necessary API keys if required.\n\n3. **Building a Chain**: LangChain is designed around the concept of chains, which are sequences of operations that take input and produce output. You can create simple chains or more complex ones depending on your use case.\n\n4. **Using the Chain**: Once you have your chain set up, you can call it with input data. LangChain will process the input through the defined steps and return the output.\n\n5. **Explore Advanced Features**: LangChain also offers advanced capabilities like memory, agents, and document loaders. These features can help you bu

In [13]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 24, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-8d5f13f5-286a-47e7-81f7-c1fb840ba70a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 9, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})